# Notebook for Feature Engineering

## Data Loading

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import colorsys
import os
import random
from tqdm import tqdm

In [1]:
# dataset info
def load_dataset(data_dir, cl_limit=20):
    X = []
    Y = []
    idx_to_cl = {}
    for i, cl in enumerate(os.listdir(data_dir)):
        if i >= cl_limit:
            break
        
        print(cl)
        idx_to_cl[i] = cl
        for fname in tqdm(os.listdir(os.path.join(data_dir,cl))):
            X.append(cv2.imread(os.path.join(data_dir, cl, fname)))
            Y.append(i)
    return np.array(X), np.array(Y), idx_to_cl


In [3]:
X, Y, idx_to_cl = load_dataset('data')

Airport


100%|██████████| 360/360 [00:05<00:00, 69.37it/s]


BareLand


100%|██████████| 310/310 [00:04<00:00, 77.21it/s] 


BaseballField


...

100%|██████████| 290/290 [00:04<00:00, 71.24it/s]


StorageTanks


100%|██████████| 360/360 [00:05<00:00, 67.84it/s]


Viaduct


100%|██████████| 420/420 [00:05<00:00, 78.35it/s]


## Feature Extraction

### HSV features
Use average and variance of 3 channels as features

In [4]:
# takes dataset and feature dictionaries as input
# gets the mean and variance of each hsv channel
def extract_hsv_features(img):
    img_convert = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    channel_mean = img_convert.mean(axis=(0,1))
    channel_var = img_convert.var(axis=(0,1))
    return np.append(channel_mean, channel_var)

### HOG vector

In [5]:
def extract_hog_features(
        img, 
        cell_size=(8,8),
        block_size=(16,16),
        block_stride=(8,8),
        num_bins=10):
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(img_gray.shape, block_size, block_stride, cell_size, num_bins)
    return hog.compute(img_gray)

### Dataset Load Function
Takes in list of feature functions and applies them within the load loop to save resources

In [6]:
# dataset info
def load_dataset(data_dir, feature_functions=[]):
    X = []
    Y = []
    idx_to_cl = {}
    
    features = []
    for i, cl in enumerate(os.listdir(data_dir)):
        print(cl)
        idx_to_cl[i] = cl
        for fname in tqdm(os.listdir(os.path.join(data_dir,cl))):
            img = cv2.imread(os.path.join(data_dir, cl, fname))
            X.append(img)
            Y.append(i)

            feature = np.array([])
            for func in feature_functions:
                feature = np.append(feature, func(img))
            features.append(feature)
            
    return np.array(X), np.array(Y), idx_to_cl, np.array(features)


In [11]:
X, Y, idx_to_cl, features = load_dataset('data', feature_functions=[extract_hsv_features,extract_hog_features])

Airport


100%|██████████| 360/360 [00:15<00:00, 22.70it/s]


BareLand


100%|██████████| 310/310 [00:12<00:00, 24.63it/s]


BaseballField


...

100%|██████████| 290/290 [00:44<00:00,  6.54it/s]


StorageTanks


100%|██████████| 360/360 [02:16<00:00,  2.63it/s]


Viaduct


100%|██████████| 420/420 [02:36<00:00,  2.69it/s]


### Orchestration

In [10]:
print(X.shape)
print(Y.shape)
print(features.shape)

(10000, 600, 600, 3)
(10000,)
(10000, 0)


In [29]:
np.array(data['Airport']).shape

(360, 600, 600, 3)

In [31]:
np.append(np.array(data['Airport']),np.array(data['BareLand']), axis=0).shape

(670, 600, 600, 3)

In [ ]:
features = {}
extract_hsv_features(dataset, features)
